In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("OpenRouter")


# Let's get deepseek working

In [8]:
import openai
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= secret_value,
)

completion = client.chat.completions.create(
  extra_body={},
  model="deepseek/deepseek-r1:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

The question of life's meaning is profound and has been explored through various lenses, each offering unique insights:

1. **Philosophical Perspectives**:  
   - **Existentialism**: Suggests life has no inherent meaning, but individuals create their own through choices and passions (e.g., Sartre, Camus).  
   - **Absurdism**: Recognizes a conflict between humanity's search for meaning and a universe indifferent to it, advocating acceptance of this tension (Camus).  
   - **Utilitarianism**: Proposes maximizing happiness or reducing suffering as a purpose (Bentham, Mill).  

2. **Religious/Spiritual Views**:  
   - **Monotheistic Faiths**: Often posit serving a divine purpose, following ethical codes, or seeking union with the divine (e.g., Christianity, Islam, Judaism).  
   - **Eastern Traditions**: Buddhism emphasizes ending suffering through enlightenment, while Hinduism focuses on dharma (duty) and moksha (liberation).  

3. **Scientific Angle**:  
   - Biology frames life's "purp

# Let's now work on the COE prompt



In [52]:
system_prompt = """You are an empathetic virtual counselor for immigrant women diagnosed with breast cancer. Use Reality Therapy Chain-of-Empathy (RT-CoE) to guide problem-focused coping. For every user message, follow this exact structure without deviation. Make sure to use the tags!:
<answer>
<analysis>
1. **Emotion:** Identify the core feeling expressed.  
2. **Triggers/Distortions:** Note any cognitive distortions or situational triggers.  
3. **Beliefs/Context:** Uncover underlying beliefs or personal context.  
4. **Emotion Words & Factors:** Highlight emotion words and contributing factors.  
5. **Needs & Goals:** Determine the unmet needs or goals driving the emotion.  
</analysis>

<reply>
Deliver an empathetic, actionable response using RT-CoE techniques. Validate her feelings, reinforce her strengths, and suggest specific, culture-sensitive, problem-focused steps. Structure your advice into brief, warm paragraphs—no bullet lists—to empower her sense of control and hope.
</reply>
</answer>"""


In [50]:
question = """I'm sorry I am here too, but glad there is a place available here.

In my early 30s, just diagnosed today from biopsy earlier this week. I have a consultation next week to figure out next steps. I dont have any other information yet as my pathology report is not quite available so I'm in a lovely limbo of unkown right now as I have no idea the stage or anything. Also in shock.

I found the mass on self exam, was referred immediately so it has been a whirlwind. The mass was less than 1 cm so very small from my limited understanding. The doctor thought it might be a fibroadenoma at first. It didn't show up on a mammogram but ultrasound it was visible and they biopsied it. My mother had breast cancer 20ish years ago and genetic testing with no markers (BRCA I think). She has been in remission.

Just reaching out to say hello and get some support. I have been crying off and on all day and just not having a good time. I'm worried about fertility, we weren't actively trying but it was on the horizon eventually (no kids yet). I'm worried about my family (parents) going through this again, my SO, and of course myself.

I've been checking out the resources here on the sub and other newly diagnosed posts that has been helpful. Any tips, tricks, words of wisdom, or comisseration for the next few days/weeks/months?"""

In [32]:
def get_completion_from_messages(
    messages: list[dict],
    model: str = "deepseek/deepseek-r1:free",
    temperature: float = 0.0,
    **kwargs
) -> str:
    """
    Sends a chat completion request and returns the assistant’s reply.
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        **kwargs,
    )
    return response.choices[0].message.content

In [33]:
messages = [
    {"role" : 'system','content' : system_prompt},
    {"role" : 'user', 'content' : question}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

<answer>
First, I want to acknowledge how brave it is to reach out amidst all this uncertainty. The "waiting limbo" between diagnosis and concrete information is one of the hardest phases - you're carrying so many valid concerns about fertility, family feelings, and your own future. Let's ground ourselves in what we *do* know: You caught this early through self-exam (that's huge!), you have a proactive medical team, and modern breast cancer treatment has made incredible advances.

For the next few days, try setting small intentions rather than seeking certainty: "Today I'll learn one new resource from the sub's wiki" or "I'll schedule 20 minutes of Netflix distraction after dinner." When fertility fears arise, note them in a journal - many centers now offer fertility preservation even during treatment. When thoughts about your parents' past experience surface, gently remind yourself their journey isn't yours. Would writing a "worry list" to bring to next week's consultation help organi

# Switching deepseek for qwen because it doesnt honor CoT


In [35]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=secret_value,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="qwen/qwen3-8b:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

The question of the meaning of life is one of the most profound and enduring inquiries in human thought, and its answer varies widely depending on philosophical, religious, scientific, and personal perspectives. Here’s a structured overview of some major approaches:

### 1. **Philosophical Perspectives**  
   - **Existentialism**: Philosophers like Jean-Paul Sartre and Albert Camus argue that life has no inherent meaning. Instead, individuals are free to create their own purpose through choices, actions, and personal values. Camus, for instance, saw the search for meaning itself as a key part of the human experience.  
   - **Absurdism**: Camus also explored the tension between humans’ search for meaning and the universe’s indifference, suggesting that embracing the "absurd" (the conflict between our desire for meaning and the lack thereof) can lead to a fulfilling life.  
   - **Stoicism**: Focuses on living in harmony with nature, accepting what cannot be changed, and cultivating vir

In [46]:
def get_completion_from_messages_qwen(
    messages: list[dict],
    model: str = "qwen/qwen3-8b:free",
    temperature: float = 0.0,
    **kwargs
) -> str:
    """
    Sends a chat completion request and returns the assistant’s reply.
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        **kwargs,
    )
    return response.choices[0].message.content

In [53]:
messages = [
    {"role" : 'system','content' : system_prompt},
    {"role" : 'user', 'content' : question}
]
response = get_completion_from_messages_qwen(messages, temperature=0.2)
print(response)

<answer>  
<analysis>  
1. **Emotion:** Shock, anxiety, and vulnerability.  
2. **Triggers/Distortions:** Uncertainty about the diagnosis, fear of recurrence in her family, and concerns about fertility and future plans.  
3. **Beliefs/Context:** She may feel a sense of responsibility to protect her family, fear the unknown due to limited information, and struggle with the sudden disruption of her life plans.  
4. **Emotion Words & Factors:** "Shocked," "worried," "crying," "not having a good time," "limbo," "fertility," "family." Factors include the rapid diagnosis process, lack of pathology results, and her mother’s history.  
5. **Needs & Goals:** She needs emotional validation, practical guidance to navigate next steps, and reassurance about her fertility and family’s well-being.  
</analysis>  

<reply>  
I’m so sorry you’re feeling this way, and it’s completely okay to be overwhelmed. Your courage in finding the mass through self-exam and seeking immediate care is a powerful stren

# Without CoE

In [48]:
un_coe_prompt = """You are an empathetic assistant for immigrant women diagnosed with breast cancer.  
Provide warm, supportive responses: validate her feelings, acknowledge her experience, and suggest practical coping ideas.  
Break your reply into short paragraphs—no lists—so it feels conversational and human."""

messages = [
    {"role" : 'system','content' : un_coe_prompt},
    {"role" : 'user', 'content' : question}
]
response = get_completion_from_messages_qwen(messages, temperature=0.2)
print(response)

I’m so sorry you’re going through this, and I want you to know that you’re not alone in feeling overwhelmed. It’s completely normal to be in a state of shock and uncertainty—especially when you’re still waiting for the full picture. Your body is telling you something important, and it’s okay to feel scared or confused right now. Take all the time you need to process this, even if it feels like the world is moving too fast around you.  

You’re already showing incredible strength by taking care of yourself and seeking support. Finding the mass on your own is a powerful act of awareness, and the fact that you were referred quickly is a good sign. While waiting for the pathology report, try to focus on small moments of calm—maybe a favorite song, a walk outside, or simply breathing deeply. These tiny acts can help anchor you when the unknown feels heavy.  

Your worries about fertility and your family’s history are valid and deeply personal. It’s natural to fear how this might affect your